In [1]:
!pip install langchain_openai langchain-neo4j

In [2]:
import warnings
import json
from langchain_neo4j import Neo4jGraph

# 실행 중 발생하는 모든 경고 메시지를 무시
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

url = "bolt://localhost:7687"  # Neo4j URI
username = "neo4j"  # Neo4j 사용자명
password = " "  # Neo4j 비밀번호 입력
database = "neo4j" # Neo4j 데이터베이스

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력

In [4]:
# Neo4j 데이터베이스에 연결
graph = Neo4jGraph(
    url=url, username=username, password=password, database=database
)

graph.refresh_schema()
schema=graph.schema

In [5]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [6]:
prompt_template = """
천천히 단계별로 생각해 봅시다:
1단계: 작업(Task)
목표:
그래프 데이터베이스에서 질의할 효과적이고 간결한 Cypher 쿼리를 생성합니다.
쿼리는 256자 이하여야 합니다.
코드에 주석을 달지 마세요.

2단계: 데이터베이스 스키마 이해하기
제공된 데이터베이스 스키마: {schema}
데이터베이스에 어떤 노드, 관계, 속성이 있는지 확인합니다.

3단계: 생성 규칙
Cypher 쿼리를 작성할 때 반드시 지켜야 할 규칙:

쿼리에서 반드시 제공된 스키마에 있는 관계 유형과 속성만 사용하세요.
사용자 질문에 나와 있더라도, 스키마에 없는 관계 또는 속성을 사용하지 마세요.
나이(Age) 관련 규칙
절대 나이를 직접 사용하지 마세요.
예: "24세" → 사용하지 말 것, 대신 "20세 이상"과 같은 범위를 사용하세요.
나이 비교에서는 항상 '초과'만 사용 (greater than)
'이하(less than)' 또는 '같음(equal)'을 사용하지 마세요.
데이터베이스에 없는 속성(Property)은 절대 사용하지 마세요.

4단계: Cypher 쿼리 예제
아래는 특정 질문에 대한 Cypher 쿼리 예제입니다.

4.1 고혈압을 유발하는 질병은?
MATCH (d:Disease)-[:HAS_HEALTH_INDICATOR]->(bp:BloodPressure)
WHERE bp.Level = 'High'
RETURN d.Name

4.2 고혈압 지표를 가진 질병은?
MATCH (d:Disease)-[:HAS_HEALTH_INDICATOR]->(bp:BloodPressure)
MATCH (d)-[:HAS_OUTCOME]->(o:Outcome)
WHERE bp.Level = 'High' AND o.Result = 'Positive'
RETURN d, properties(d) AS disease_properties, bp, properties(bp) AS blood_pressure_properties, o, properties(o) AS outcome_properties


4.3 고혈압, 고콜레스테롤, 발열, 피로 증상을 보이는 노인 질병은?
MATCH (d:Disease)-[:HAS_HEALTH_INDICATOR]->(bp:BloodPressure)
MATCH (d)-[:HAS_HEALTH_INDICATOR]->(ch:CholesterolLevel)
MATCH (d)-[:HAS_SYMPTOM]->(f:Fever)
MATCH (d)-[:HAS_SYMPTOM]->(fa:Fatigue)
MATCH (d)-[:HAS_DEMOGRAPHIC]->(a:Age)
WHERE bp.Level = 'High'
  AND ch.Level = 'High'
  AND f.Status = 'Yes'
  AND fa.Status = 'Yes'
  AND a.Value > 40
RETURN d.Name


4.4 고콜레스테롤이 있는 환자가 발열과 피로를 보이며 기침, 호흡곤란이 없는 경우 유발하는 질병은?
MATCH (d:Disease)-[:HAS_SYMPTOM]->(f:Fever)
MATCH (d)-[:HAS_SYMPTOM]->(fa:Fatigue)
MATCH (d)-[:HAS_SYMPTOM]->(c:Cough)
MATCH (d)-[:HAS_SYMPTOM]->(db:DifficultyBreathing)
MATCH (d)-[:HAS_HEALTH_INDICATOR]->(ch:CholesterolLevel)
MATCH (d)-[:HAS_OUTCOME]->(o:Outcome)
WHERE f.Status = 'Yes'
  AND fa.Status = 'Yes'
  AND c.Status = 'No'
  AND db.Status = 'No'
  AND ch.Level = 'High'
  AND o.Result = 'Positive'
RETURN d, properties(d) AS disease_properties, ch, properties(ch) AS cholesterol_properties, o, properties(o) AS outcome_properties


5단계: 각 속성에 대한 허용값
발열(Fever): 환자가 발열이 있는지 여부 (Yes/No)
기침(Cough): 환자가 기침이 있는지 여부 (Yes/No)
피로(Fatigue): 환자가 피로를 느끼는지 여부 (Yes/No)
호흡곤란(Difficulty Breathing): 환자가 호흡곤란이 있는지 여부 (Yes/No)
나이(Age): 환자의 나이 (20세 이상 등 범위로 사용)
성별(Gender): 환자의 성별 (Male/Female)
혈압(Blood Pressure): 혈압 수준 (Normal/High)
콜레스테롤 수치(Cholesterol Level): 콜레스테롤 수준 (Normal/High)
질병 결과(Outcome Variable): 질병 진단 결과 (Positive/Negative)

6단계: 사용자의 질문을 Cypher 쿼리로 변환
{question}을 기반으로 위의 규칙을 적용하여 Cypher 쿼리를 생성하세요."""

In [9]:
# 랭체인 PromptTemplate을 사용하여 Cypher 쿼리를 생성할 템플릿을 설정
from langchain.prompts.prompt import PromptTemplate
cypher_prompt = PromptTemplate(
    input_variables=["schema","question"], 
    template=prompt_template
)

# 랭체인을 사용하여 자연어 질문을 Cypher 쿼리로 변환 후, Neo4j에서 검색하는 체인 생성
from langchain_neo4j import GraphCypherQAChain
cypherChain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    cypher_prompt=cypher_prompt, # Cypher 쿼리를 생성할 때 사용할 프롬프트 
    allow_dangerous_requests = True,
    top_k=10 # 검색 결과에서 최대 10개의 문서를 반환
)

In [15]:
cypherChain.invoke("""45세 남성이고, 고혈압, 발열, 호흡곤란 증상이 있어. 가능한 질환은?""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Disease)-[:HAS_HEALTH_INDICATOR]->(bp:BloodPressure)
MATCH (d)-[:HAS_SYMPTOM]->(f:Fever)
MATCH (d)-[:HAS_SYMPTOM]->(db:DifficultyBreathing)
MATCH (d)-[:HAS_DEMOGRAPHIC]->(a:Age)
WHERE bp.Level = 'High'
  AND f.Status = 'Yes'
  AND db.Status = 'Yes'
  AND a.Value > 45
RETURN d.Name

Full Context:
[{'d.Name': 'Pneumonia'}]

> Finished chain.


{'query': '45세 남성이고, 고혈압, 발열, 호흡곤란 증상이 있어. 가능한 질환은?',
 'result': '폐렴이 가능한 질환입니다.'}

In [16]:
cypherChain.invoke("""나이는 30세이고 남성이야, 고열, 피로, 기침 증상이 있어. 가능한 질환은?""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Disease)-[:HAS_SYMPTOM]->(f:Fever)
MATCH (d)-[:HAS_SYMPTOM]->(fa:Fatigue)
MATCH (d)-[:HAS_SYMPTOM]->(c:Cough)
MATCH (d)-[:HAS_DEMOGRAPHIC]->(a:Age)
WHERE f.Status = 'Yes'
  AND fa.Status = 'Yes'
  AND c.Status = 'Yes'
  AND a.Value > 30
RETURN d.Name

Full Context:
[{'d.Name': 'Hypertension'}, {'d.Name': 'Pneumonia'}]

> Finished chain.


{'query': '나이는 30세이고 남성이야, 고열, 피로, 기침 증상이 있어. 가능한 질환은?',
 'result': '고열, 피로, 기침 증상에 대한 가능한 질환은 고혈압과 폐렴입니다.'}

In [18]:
cypherChain.invoke("""나이는 55세이고 남성이야, 고혈압에 고콜레스테롤, 발열 증상이 있어. 가능한 질환은?""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Disease)-[:HAS_HEALTH_INDICATOR]->(bp:BloodPressure)
MATCH (d)-[:HAS_HEALTH_INDICATOR]->(ch:CholesterolLevel)
MATCH (d)-[:HAS_SYMPTOM]->(f:Fever)
MATCH (d)-[:HAS_DEMOGRAPHIC]->(a:Age)
MATCH (d)-[:HAS_DEMOGRAPHIC]->(g:Gender)
WHERE bp.Level = 'High'
  AND ch.Level = 'High'
  AND f.Status = 'Yes'
  AND a.Value > 55
  AND g.Type = 'Male'
RETURN d.Name

Full Context:
[]

> Finished chain.


{'query': '나이는 55세이고 남성이야, 고혈압에 고콜레스테롤, 발열 증상이 있어. 가능한 질환은?',
 'result': "I don't know the answer."}